In [30]:
import networkx
import random as rd
import numpy as np

In [3]:
def barabasi(total_n, starting_n, directed = False):
    ### Generate a Barabasi-Albert network simulation
    # total_n: total number of nodes
    # starting_n: number of nodes to generate edges uniformly at random
    # directed: if not directed will include reverse edges
    # returns edges: list of tuples (i,j) where i is connected to j
    ###
    edges = []
    for i in range(starting_n):
        # Generate links uniformly at random
        temp = [(i,j) for j in range(starting_n) if rd.uniform(0,1) < 1/2 and i != j]
        edges.extend(temp)
        # If not directed then generate the reverse links
        if directed == False:
            temp1 = [(j,i) for (i,j) in temp]
            edges.extend(temp1)
    for l in range(starting_n, total_n):
        n = len(edges)
        if directed == False:
            n = n/2
        # Compute the probability for a new node to connect to each existing node
        temp_probs = [len(list(filter(lambda x: j == x[1], edges)))/n for (i,j) in edges]
        # Generate links for a new node
        temp = [(l,j) for j in range(l) if rd.uniform(0,1) < temp_probs[j] and l != j]
        edges.extend(temp)
        if directed == False:
            temp1 = [(j,i) for (i,j) in temp]
            edges.extend(temp1)
    return edges

In [31]:
def postpublish(friends_edges, follower_edges, m0, m1):
    ### Simulate a list of posts and users who create them (as a tuple)
    # friends_edges: list of edges as tuples for friendship network
    # follower_edges: list of edges as tuples for follower network
    # m0: starting number of posts
    # m1: total number of posts
    # returns postpublished: list of tuples (i,j) where user i posted post j
    ###
    postpublished = []
    
    # Get the max edges
    temp = [i for (i,j) in friends_edges]
    temp.extend([j for (i,j) in friends_edges])
    n = max(temp)
    
    # Generate the first posts
    for i in range(m0):
        postpublished.append((rd.randint(0,n),i))
    
    # Generate the rest of the posts
    for m in range(m0,m1):
        # Compute the prob that anyone posts
        probs = []
        for k in range(0,n):
            temp_probs = [len(list(filter(lambda x: j == x[1], friends_edges)))
                                  for (i,j) in friends_edges]
            temp_probs = temp_probs + [len(list(filter(lambda x: j == x[1], follower_edges)))
                                  for (i,j) in follower_edges]
            temp_probs = temp_probs + [len(list(filter(lambda x: i == x[0], postpublished)))
                                  for (i,j) in postpublished]
            probs.append(temp_probs)
        temp_user = rd.choices(list[range(0,n)], weights = probs)
        postpublished.append((temp_user,m))                      
    
    return postpublished
    

In [4]:
network=barabasi(10,5)

In [25]:
rd.randint(0,10)

8

In [29]:
lists = [(7,9),(0,10)]
lists[1]./7

SyntaxError: invalid syntax (<ipython-input-29-92973fe0490f>, line 2)

In [20]:
aaa= [i for (i,j) in network]

In [22]:
aaa.extend([j for (i,j) in network])

In [23]:
aaa

[0,
 0,
 0,
 1,
 2,
 3,
 1,
 0,
 2,
 2,
 0,
 4,
 3,
 4,
 4,
 4,
 2,
 3,
 5,
 5,
 3,
 4,
 6,
 6,
 6,
 1,
 3,
 5,
 7,
 7,
 7,
 7,
 1,
 2,
 4,
 5,
 8,
 8,
 3,
 5,
 9,
 4,
 1,
 2,
 3,
 0,
 0,
 0,
 0,
 1,
 0,
 4,
 2,
 2,
 4,
 3,
 2,
 3,
 4,
 4,
 3,
 4,
 5,
 5,
 1,
 3,
 5,
 6,
 6,
 6,
 1,
 2,
 4,
 5,
 7,
 7,
 7,
 7,
 3,
 5,
 8,
 8,
 4,
 9]